# SubSection Management

## Setup

### Imports

In [4]:
from pprint import pprint
from sections import Section
from sections import SectionBreak
from buffered_iterator import BufferedIterator
from sections import set_method
from sections import ProcessingMethods

### Test Text

In [5]:
test_text = [
            'Text to be ignored',
            'StartSection A',
            'MiddleSection A',
            'EndSection A',
            'Unwanted text between sections',
            'StartSection B',
            'MiddleSection B',
            'EndSection B',
            'StartSection C',
            'MiddleSection C',
            'EndSection C',
            'Even more text to be ignored',
            ]

multi_section_text = [
    'StartSection Name:A',
    'A Content1:a',
    'EndSection Name:A',
    'StartSection Name:B',
    'A Content2:a',
    'EndSection Name:B'
    ]

## Context updating

Test external context updating

### Context update in function
![Good](../Valid.png) test_context is being updated

In [6]:
def set_context(item, context):
    context['This Item'] = item
    return item


test_context = {'dummy': 'Test'}

set_context('A', test_context)

test_context

{'dummy': 'Test', 'This Item': 'A'}

### Context update in *set_method* function
![Good](../Valid.png) test_context is being updated

In [7]:
def set_context(item, context):
    context['This Item'] = item
    return item


use_function = set_method(set_context, method_type='Process')

test_context = {'dummy': 'Test'}

use_function('A', test_context)

test_context

{'dummy': 'Test', 'This Item': 'A'}

### Context update in ProcessingMethods
![Good](../Valid.png) test_context is being updated

In [8]:
def set_context(item, context):
    context['This Item'] = item
    return item

multi_section_text = [
    'StartSection Name:A',
    'A Content1:a',
    'EndSection Name:A',
    'StartSection Name:B',
    'A Content2:a',
    'EndSection Name:B'
    ]

test_iter = BufferedIterator(multi_section_text)
test_context = {'dummy': 'Test'}

processor = ProcessingMethods([set_context])

pprint(processor.read(test_iter, test_context))

test_context

['StartSection Name:A',
 'A Content1:a',
 'EndSection Name:A',
 'StartSection Name:B',
 'A Content2:a',
 'EndSection Name:B']


{'dummy': 'Test', 'This Item': 'EndSection Name:B'}

### Context update with *section.processor.reader*
![Good](../Valid.png) test_context is being updated

In [9]:
def set_context(item, context):
    context['This Item'] = item
    return item

multi_section_text = [
    'StartSection Name:A',
    'A Content1:a',
    'EndSection Name:A',
    'StartSection Name:B',
    'A Content2:a',
    'EndSection Name:B'
    ]

test_iter = BufferedIterator(multi_section_text)
test_context = {'dummy': 'Test'}

content_section = Section(
    name='Content',
    end_section=SectionBreak('EndSection', break_offset='Before', name='EndContent'),
    processor=set_context
    )

section_proc = content_section.processor.reader(test_iter, context = test_context)

pprint([i for i in section_proc])

print('test_context', test_context)


['StartSection Name:A',
 'A Content1:a',
 'EndSection Name:A',
 'StartSection Name:B',
 'A Content2:a',
 'EndSection Name:B']
test_context {'dummy': 'Test', 'This Item': 'EndSection Name:B'}


### Context update with *section.scan* and *section.processor.reader*
![Good](../Valid.png) test_context is being updated


In [10]:
def set_context(item, context):
    context['This Item'] = item
    return item

multi_section_text = [
    'StartSection Name:A',
    'A Content1:a',
    'EndSection Name:A',
    'StartSection Name:B',
    'A Content2:a',
    'EndSection Name:B'
    ]

test_iter = BufferedIterator(multi_section_text)
test_context = {'dummy': 'Test'}


content_section = Section(
    name='Content',
    end_section=SectionBreak('EndSection', break_offset='Before', name='EndContent'),
    processor=set_context
    )


section_iter = content_section.scan(test_iter, context = test_context)
read_iter = content_section.processor.reader(section_iter, context = test_context)

pprint([i for i in read_iter])

test_context

['StartSection Name:A', 'A Content1:a']


{'dummy': 'Test', 'This Item': 'A Content1:a'}

### Context update with *ProcessingMethods* using two functions
![Good](../Valid.png) 
test_context is being propagated even when the second processing function 
doesn't take context.

In [11]:
def set_context(item, context):
    context['This Item'] = item
    return item

def do_nothing(item):
    return item

multi_section_text = [
    'StartSection Name:A',
    'A Content1:a',
    'EndSection Name:A',
    'StartSection Name:B',
    'A Content2:a',
    'EndSection Name:B'
    ]

test_iter = BufferedIterator(multi_section_text)
test_context = {'dummy': 'Test'}

processor = ProcessingMethods([set_context, do_nothing])

pprint(processor.read(test_iter, test_context))

test_context

['StartSection Name:A',
 'A Content1:a',
 'EndSection Name:A',
 'StartSection Name:B',
 'A Content2:a',
 'EndSection Name:B']


{'dummy': 'Test', 'This Item': 'EndSection Name:B'}

### Context update with *section.scan*, *section.processor.reader* and *list()*
![Good](../Valid.png) 
test_context is being updated when the list() function is called

In [12]:
def set_context(item, context):
    context['This Item'] = item
    return item

multi_section_text = [
    'StartSection Name:A',
    'A Content1:a',
    'EndSection Name:A',
    'StartSection Name:B',
    'A Content2:a',
    'EndSection Name:B'
    ]

test_iter = BufferedIterator(multi_section_text)
test_context = {'dummy': 'Test'}


content_section = Section(
    name='Content',
    end_section=SectionBreak('EndSection', break_offset='Before', name='EndContent'),
    processor=set_context
    )


section_iter = content_section.scan(test_iter, context = test_context)
read_iter = content_section.processor.reader(section_iter, context = test_context)

#pprint([i for i in read_iter])
list(read_iter)

test_context

{'dummy': 'Test', 'This Item': 'A Content1:a'}

 ### Context update with *section.scan*, *section.processor.reader* and *section.assemble*
![Good](../Valid.png) 
test_context is being updated when the section.assemble() function is called

In [13]:
def set_context(item, context):
    context['This Item'] = item
    return item

multi_section_text = [
    'StartSection Name:A',
    'A Content1:a',
    'EndSection Name:A',
    'StartSection Name:B',
    'A Content2:a',
    'EndSection Name:B'
    ]

test_iter = BufferedIterator(multi_section_text)
test_context = {'dummy': 'Test'}


content_section = Section(
    name='Content',
    end_section=SectionBreak('EndSection', break_offset='Before', name='EndContent'),
    processor=set_context
    )


section_iter = content_section.scan(test_iter, context = test_context)
read_iter = content_section.processor.reader(section_iter, context = test_context)

section_assembled = content_section.assemble(read_iter, context=test_context)

pprint(section_assembled)
print('test_context', test_context)


['StartSection Name:A', 'A Content1:a']
test_context {'dummy': 'Test', 'This Item': 'A Content1:a'}


 ### Context update with *section.read*
 
![Good](../Valid.png) When the section.read() function is called
- test_context is not being updated 
- content_section.context is being updated 

In [14]:
def set_context(item, context):
    context['This Item'] = item
    return item

multi_section_text = [
    'StartSection Name:A',
    'A Content1:a',
    'EndSection Name:A',
    'StartSection Name:B',
    'A Content2:a',
    'EndSection Name:B'
    ]

test_iter = BufferedIterator(multi_section_text)
test_context = {'dummy': 'Test'}


content_section = Section(
    name='Content',
    end_section=SectionBreak('EndSection', break_offset='Before', name='EndContent'),
    processor=set_context
    )


assembled_section = content_section.read(test_iter, context = test_context)

print(assembled_section,'\n')
print('test_context', test_context,'\n')
print('content_section.context')
pprint(content_section.context, indent=4)

['StartSection Name:A', 'A Content1:a'] 

test_context {'dummy': 'Test'} 

content_section.context
{   'Break': 'EndContent',
    'Current Section': 'Content',
    'Event': 'EndSection',
    'Skipped Lines': [],
    'Status': 'Break Triggered',
    'This Item': 'A Content1:a'}


### Processing function that reads from context


In [ ]:
def set_context(item, context):
    name = context.get('Current Section')
    if name:                               
        context[name] = item
    else:
        context['No section'] = item
    return item


def print_context(item, context):
    pprint(context)
    return item

test_text = [
            'Text to be ignored',
            'StartSection A',
            'MiddleSection A',
            'EndSection A',
            'Unwanted text between sections',
            'StartSection B',
            'MiddleSection B',
            'EndSection B',
            'StartSection C',
            'MiddleSection C',
            'EndSection C',
            'Even more text to be ignored',
            ]


test_iter = BufferedIterator(test_text)

single_section = Section(
    name='3-part Section',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    processor=set_context
    )

while True:
    if single_section.scan_status in ['Scan Complete', 'End of Source']:
        break
    assembled_section = single_section.read(test_iter)
    print(assembled_section )

print('single_section.context')
pprint(single_section.context, indent=4)



['StartSection A', 'MiddleSection A', 'EndSection A']
['StartSection B', 'MiddleSection B', 'EndSection B']
['StartSection C', 'MiddleSection C', 'EndSection C']
[]
single_section.context
{   '3-part Section': 'EndSection C',
    'Current Section': '3-part Section',
    'Skipped Lines': ['Even more text to be ignored'],
    'Status': 'End of Source'}


### No context supplied
 
![Good](../Valid.png) When the section.read() function is called
- test_context is not being updated 
- content_section.context is being updated 

In [15]:
def set_context(item, context):
    context['This Item'] = item
    return item

multi_section_text = [
    'StartSection Name:A',
    'A Content1:a',
    'EndSection Name:A',
    'StartSection Name:B',
    'A Content2:a',
    'EndSection Name:B'
    ]

test_iter = BufferedIterator(multi_section_text)
test_context = None

content_section = Section(
    name='Content',
    end_section=SectionBreak('EndSection', break_offset='Before', name='EndContent'),
    processor=set_context
    )


assembled_section = content_section.read(test_iter)

print(assembled_section,'\n')
print('test_context:\t', test_context,'\n')
pprint(content_section.context)

['StartSection Name:A', 'A Content1:a'] 

test_context:	 None 

{'Break': 'EndContent',
 'Current Section': 'Content',
 'Event': 'EndSection',
 'Skipped Lines': [],
 'Status': 'Break Triggered',
 'This Item': 'A Content1:a'}


## Subsection Requirements
### Context
- Do not change the supplied context
- Use *self.context*
- *self.context* needs to be updated after every stage
- Need to protect standard context items from being changed by sub sections

### Source
- Supplied source needs to be isolated from the iterated source or it may exit too soon.
- Section's source pointer needs to be adjusted so that any "Future Items" are not missed.
- Both context and iterator should be isolated by default 


# Multiple SubSections

## Single line subsections

### Check individual subsections

In [38]:
multi_section_text = [
    'StartSection Name:A',
    'A Content1:a',
    'EndSection Name:A',
    'StartSection Name:B',
    'A Content2:a',
    'EndSection Name:B'
    ]

name_section = Section(
    name='Name',
    end_section=SectionBreak(True, name='NameEnd')
    )
content_section = Section(
    name='Content',
    end_section=SectionBreak('EndSection', break_offset='Before', name='EndContent')
    )
end_section = Section(
    name='End',
    end_section=SectionBreak(True, name='EndEnd')
    )

test_iter = BufferedIterator(multi_section_text)

print(name_section.read(test_iter))
print(content_section.read(test_iter))
print(end_section.read(test_iter))

['StartSection Name:A']
['A Content1:a']
['EndSection Name:A']


### List of subsections

In [40]:
multi_section_text = [
    'StartSection Name:A',
    'A Content1:a',
    'EndSection Name:A',
    'StartSection Name:B',
    'A Content2:a',
    'EndSection Name:B'
    ]

name_section = Section(
    name='Name',
    end_section=SectionBreak(True, name='NameEnd')
    )

content_section = Section(
    name='Content',
    end_section=SectionBreak('EndSection', 
                             break_offset='Before', 
                             name='EndContent')
    )

end_section = Section(
    name='End',
    end_section=SectionBreak(True, name='EndEnd')
    )

test_iter = BufferedIterator(multi_section_text)
section_list = [name_section, content_section, end_section]

combined_sections = []
while True:
    section_group = {}
    for sub_section in section_list:
        sub_section_item = sub_section.read(test_iter)
        section_group[sub_section.name] = sub_section_item
    combined_sections.append(section_group)
    if sub_section.scan_status in ['Scan Complete', 'End of Source']:
        break
    

pprint(combined_sections)


[{'Content': ['A Content1:a'],
  'End': ['EndSection Name:A'],
  'Name': ['StartSection Name:A']},
 {'Content': ['A Content2:a'],
  'End': ['EndSection Name:B'],
  'Name': ['StartSection Name:B']}]


# Done to Here

TODO Write a function to handle a list of objects containing .context and .name attributes and a .read() method

In [ ]:

combined_sections = []
while True:
    section_group = {}
    for sub_section in section_list:
        sub_section_item = sub_section.read(test_iter)
        section_group[sub_section.name] = sub_section_item
    combined_sections.append(section_group)
    if sub_section.scan_status in ['Scan Complete', 'End of Source']:
        break
    


In [ ]:
name_section = Section(
    name='SingleSection',
    start_section=SectionBreak('StartSection', name='StartSection'),
    end_section=SectionBreak('EndSection', name='EndSection')
    )


In [ ]:
full_section = Section(
    name='Single Subsection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    processor=[name_section, content_section, end_section]
    )

In [ ]:
multi_section_text = [
    'Text to be ignored',
    'StartSection Name:A',
    'A Content1:a',
    'B Content1:b',
    'C Content1:c',
    'EndSection Name:A',
    'StartSection Name:B',
    'A Content2:a',
    'B Content2:b',
    'C Content2:c',
    'EndSection Name:B'
    'Even more text to be ignored',
    ]

test_iter = BufferedIterator(multi_section_text)


In [ ]:
def set_context(item, context):
    name = context.get('Current Section')
    if name:                               
        context[name] = item
    else:
        context['No section'] = item
    return item


def print_context(item, context):
    pprint(context)
    return item


In [ ]:
name_section = Section(
    name='Name',
    end_section=SectionBreak(True, name='NameEnd'),
    processor=set_context
    )
content_section = Section(
    name='Content',
    end_section=SectionBreak('EndSection', break_offset='Before', name='EndContent'),
    processor=set_context
    )
end_section = Section(
    name='End',
    end_section=SectionBreak(True, name='EndEnd'),
    processor=print_context
    )


In [ ]:
full_section = Section(
    name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    processor=[name_section, content_section, end_section]
    )

In [ ]:

section_list = [name_section, content_section, end_section]

test_context = {'dummy': 'Test'}

all_sections = []
while True:
    section_group = {}
    active_context = test_context
    for sub_section in section_list:
        sub_section_item = sub_section.read(test_iter, context = test_context)
        section_group[sub_section.name] = sub_section_item
        active_context = sub_section.context
    if sub_section.scan_status in ['Scan Complete', 'End of Source']:
        break
    all_sections.append(section_group)
    

print('test_context:\t', test_context,'\n')
print('all_sections')
pprint(all_sections, indent=4)

print('sub_section.context')
pprint(sub_section.context, indent=4)

In [19]:
test_iter = BufferedIterator(multi_section_text)

section_list = [name_section, content_section, end_section]

test_context = {'dummy': 'Test'}



while True:
    section_group = {}
    for sub_section in section_list:
        sub_section_item = sub_section.read(test_iter, context = test_context)
        section_group[sub_section.name] = sub_section_item
        #pprint(sub_section.context)
        #print()
    pprint(section_group)
    if sub_section.scan_status in ['Scan Complete', 'End of Source']:
        break
pprint(sub_section.context)


{'Content': ['A Content1:a'],
 'End': ['EndSection Name:A'],
 'Name': ['StartSection Name:A']}
{'Content': ['A Content2:a'],
 'End': ['EndSection Name:B'],
 'Name': ['StartSection Name:B']}
{'Current Section': 'End', 'Skipped Lines': [], 'Status': 'End of Source'}


## set_subsection_reader
``

Converts section instances to functions that call the section reader.

Args:<br>
> **processing_def (ProcessMethodDef):** A processing method.

Raises:<br>
>  **ValueError:** For processing instruction items which are lists where
        the list items are not all Section instances.

Returns:<br>
>  **ProcessMethodDef:** If processing_def is a section object, or a list
    of section objects, return a partial function that calls the
    section(s) read_subsections method.
    Otherwise returns processing_def.